A neural network consist of cnn layer (Yoon Kim,2014) and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g75qRFjAgmxamzvYFy9fMGiGtPYCY4D_d06JCDuXtlzibGdOXbHKMw
Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [3]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0)
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [5]:
len(labels)

78

In [6]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 4404


In [7]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [8]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [9]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [10]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [11]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12896, 4404) 
test:  (4299, 4404) 
val:  (4299, 4404) 
y_tain: (12896, 78)


In [12]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [13]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [14]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [15]:

word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [16]:
from keras.layers import LSTM
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn
    cnn = Conv2D(1, 
                 kernel_size=(5, sentence_maxlen),
                 activation='relu')(embd_sentence)
    cnn =  Lambda(lambda x: K.sum(x, axis=3))(cnn)
    cnn = MaxPooling1D(3)(cnn)
    cnn = Lambda(lambda x: K.sum(x, axis=2))(cnn)
    
    hidden1=Dense(400,activation="relu")(cnn)
    hidden2=Dense(300,activation="relu")(hidden1)
    hidden3=Dense(200,activation="relu")(hidden2)
    hidden4=Dense(150,activation="relu")(hidden3)
    out = Dense(len(labels), activation='sigmoid')(hidden4)
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy',metrics=["accuracy"]) 

    return model

model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())


Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 4404)]            0         
_________________________________________________________________
shared_embd (Embedding)      (None, 4404, 300)         67603500  
_________________________________________________________________
permute (Permute)            (None, 300, 4404)         0         
_________________________________________________________________
lambda (Lambda)              (None, 300, 4404, 1)      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 296, 1, 1)         22021     
_________________________________________________________________
lambda_1 (Lambda)            (None, 296, 1)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 98, 1) 

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50) # Model stop training after 50 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=50,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss

Epoch 1/50
403/403 [==============================] - 675s 2s/step - loss: 0.6377 - accuracy: 0.0230 - val_loss: 0.5882 - val_accuracy: 0.0263

Epoch 00001: val_loss improved from inf to 0.58824, saving model to best_cnn_4fc.h5
Epoch 2/50
403/403 [==============================] - 670s 2s/step - loss: 0.5466 - accuracy: 0.0285 - val_loss: 0.5083 - val_accuracy: 0.0263

Epoch 00002: val_loss improved from 0.58824 to 0.50832, saving model to best_cnn_4fc.h5
Epoch 3/50
403/403 [==============================] - 667s 2s/step - loss: 0.4757 - accuracy: 0.0312 - val_loss: 0.4459 - val_accuracy: 0.0263

Epoch 00003: val_loss improved from 0.50832 to 0.44592, saving model to best_cnn_4fc.h5
Epoch 4/50
403/403 [==============================] - 665s 2s/step - loss: 0.4201 - accuracy: 0.0278 - val_loss: 0.3966 - val_accuracy: 0.0263

Epoch 00004: val_loss improved from 0.44592 to 0.39665, saving model to best_cnn_4fc.h5
Epoch 5/50
403/403 [==============================] - 661s 2s/step - loss: 0

# ***If the model is generated before:***

In [ ]:
# model = load_model('CNN_1_just_accuracy.h5')
# model.save('CNN_1_just_accuracy.h5')

In [ ]:
model.save('CNN_1_just_accuracy.h5')

In [ ]:
X_test.shape , X_train.shape

In [ ]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [ ]:
print(pred[0])#example

In [ ]:
y_pred=[]
measure = .23#9*(np.mean(pred[0]) + .5*np.sqrt(np.var(pred[0])))
for l in pred:
  temp=[]
  for value in l:
    if value >= measure:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [ ]:
3*(np.mean(pred[0]) + .5*np.sqrt(np.var(pred[0])))

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


## ROC Curve

In [ ]:
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(Y[0].shape[0]):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


In [ ]:
for i in range(Y[0].shape[0]):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()